In [1]:
import bs4
import requests
import re
import json
import pandas as pd

from os import path
from tqdm import tqdm_notebook
from time import sleep

In [2]:
base_url = 'https://www.cvk.gov.ua/pls/vp2019/'
page = requests.get(path.join(base_url,'wp030pt001f01=719.html')).text
soup = bs4.BeautifulSoup(page,'html.parser')

district_links = []
for item in soup.find_all('tr', class_="tr"):
    district_links.append(item.find(class_ = 'a1').get('href'))
        
district_links[:10]

['wp024npt001f01=719pid100=5pt005f01=11.html',
 'wp024npt001f01=719pid100=5pt005f01=12.html',
 'wp024npt001f01=719pid100=5pt005f01=13.html',
 'wp024npt001f01=719pid100=5pt005f01=14.html',
 'wp024npt001f01=719pid100=5pt005f01=15.html',
 'wp024npt001f01=719pid100=5pt005f01=16.html',
 'wp024npt001f01=719pid100=5pt005f01=17.html',
 'wp024npt001f01=719pid100=5pt005f01=18.html',
 'wp024npt001f01=719pid100=7pt005f01=19.html',
 'wp024npt001f01=719pid100=7pt005f01=20.html']

In [3]:
def parse_row(results):
    for row in results:
        for index,column in enumerate(row.find_all('td')[:2]):
            if index == 0:
                district_number = column.text.replace(u'\n', u'')
            elif index == 1:
                district_location = column.text.replace(u'\n', u'')
        records.append([district_general,district_number,district_location])

records = []

for district_link in tqdm_notebook(district_links):
    if district_link == 'wp045pt001f01=719.html':
        page = requests.get(path.join(base_url,district_link)).text
        soup = bs4.BeautifulSoup(page,'html5lib')
        fd_links = [link.get('href') for link in soup.find_all(class_="pure-table")[1].find_all(class_='a1')]

        for fd_link in fd_links:
            page = requests.get(path.join(base_url,fd_link)).text
            soup = bs4.BeautifulSoup(page,'html5lib')
            sleep(0.2)

            district_general = soup.find('h3').text
            results_raw = soup.find(class_='pure-table w100').find('tbody')
            parse_row(results_raw.find_all('tr'))
            
    else:
        page = requests.get(path.join(base_url,district_link)).text
        soup = bs4.BeautifulSoup(page,'html5lib')
        inner_link = soup.find(class_="fas fa-2x fa-warehouse").parent.get('href')

        page = requests.get(path.join(base_url,inner_link)).text
        soup = bs4.BeautifulSoup(page,'html5lib')
        sleep(0.2)

        district_general = soup.find(class_='pure-u-1 cntr').text.replace(u'\xa0', u'').replace(u'\n', u'')
        results_raw = soup.find(class_='pure-table pure-table-horizontal').find('tbody')
        parse_row(results_raw.find_all('tr')[1:])
    
colnames = ['district_general','district_number','district_location']
df_locations = pd.DataFrame.from_records(records,columns=colnames)
print(df_locations.shape)
df_locations.head()


(29982, 3)


,district_general,district_number,district_location
0,"Вінницька область, Територіальний виборчий окр...",050130,"Школа-ліцей, спортивна зала"
1,"Вінницька область, Територіальний виборчий окр...",050131,"ДНЗ ""Казка"", спортивна зала"
2,"Вінницька область, Територіальний виборчий окр...",050132,"ПСП ""Агрофірма Батьківщина"", приіщення контори"
3,"Вінницька область, Територіальний виборчий окр...",050133,"школа, спортзал"
4,"Вінницька область, Територіальний виборчий окр...",050134,"звіргосподарство, кабінет головного бухгалтера"


In [5]:
base_url = 'https://www.cvk.gov.ua/pls/vp2019/'
page = requests.get(path.join(base_url,'wp335pt001f01=719.html')).text
soup = bs4.BeautifulSoup(page,'html5lib')
links = [link_raw.get('href') for link_raw in soup.find_all(class_='a1')]
districts = [int(link.split('=')[-1].replace('.html','')) for link in links]
links_and_districts = [i for i in zip(districts,links)]
links_and_districts[:10]

[(11, 'wp336pt001f01=719pt005f01=11.html'),
 (12, 'wp336pt001f01=719pt005f01=12.html'),
 (13, 'wp336pt001f01=719pt005f01=13.html'),
 (14, 'wp336pt001f01=719pt005f01=14.html'),
 (15, 'wp336pt001f01=719pt005f01=15.html'),
 (16, 'wp336pt001f01=719pt005f01=16.html'),
 (17, 'wp336pt001f01=719pt005f01=17.html'),
 (18, 'wp336pt001f01=719pt005f01=18.html'),
 (19, 'wp336pt001f01=719pt005f01=19.html'),
 (20, 'wp336pt001f01=719pt005f01=20.html')]

In [6]:
colvalues = []
district_gen_numeric = []

for link in tqdm_notebook(links_and_districts):
    page = requests.get(path.join(base_url,link[1])).text
    soup = bs4.BeautifulSoup(page,'html5lib')
    results_raw = soup.find(class_='pure-table pure-table-horizontal w100')
    results_raw.find('thead').find_all('th')[0].text
    colnames = [colname.text.replace(u'\n', u'') for colname in results_raw.find('thead').find_all('th')]
    for row in results_raw.find_all('tr')[1:]:
        colvalues_row = []
        for colvalue in row.find_all('td'):
            colvalues_row.append(colvalue.text.replace(u'\n', u''))
        colvalues.append(colvalues_row)
        district_gen_numeric.append(link[0])
    sleep(0.2)

df_results = pd.DataFrame.from_records(colvalues,columns=colnames)
df_results['district_gen_numeric'] = pd.Series(district_gen_numeric)
print(df_results.shape)
df_results.head()


(29989, 52)


,№ ВД,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.","7.Загальна к-сть виборців, які отримали бюлетені","8.К-сть бюлетенів, що не підлягають врахуванню","9.К-сть виборців, які взяли участь у голосуванні",...,СкоцикВіталій,СмешкоІгор,СоловйовОлександр,ТарутаСергій,ТимошенкоЮлія,ТимошенкоЮрій,ШевченкоІгор,ШевченкоОлександр,Дата та час протоколу,district_gen_numeric
0,050130,1704,1686,112,583,1031,90,1121,0,1121,...,1,120,0,0,169,11,1,3,01.04.2019 01:30,11
1,050131,2148,2109,17,790,1348,10,1358,0,1358,...,2,147,0,2,213,19,0,9,01.04.2019 00:50,11
2,050132,1294,1282,4,453,837,4,841,0,841,...,0,95,2,0,104,9,1,1,02.04.2019 23:45,11
3,050133,2439,2434,15,845,1582,12,1594,0,1594,...,3,186,0,0,231,10,1,12,01.04.2019 19:00,11
4,050134,222,225,0,82,140,0,140,0,140,...,0,14,0,0,26,0,1,0,01.04.2019 00:10,11


In [68]:
df_locations['district_gen_numeric'] = df_locations['district_general'].map(lambda x: int(x.split('№')[-1]) if len(x.split('№')[-1])<4 else 226)
df_locations['district_number'] = df_locations.loc[:,'district_number'].map(lambda x: x.replace(' ',''))
df_all = df_results.rename(columns={'№ ВД':'district_number'}).merge(df_locations,on=['district_number','district_gen_numeric'])
df_all = df_all[['district_number','district_gen_numeric', 'district_general','district_location'] + list(df_all.columns[1:-3])]
print(df_all.shape)
df_all.head()

(29989, 54)


,district_number,district_gen_numeric,district_general,district_location,1.К-сть одержаних бюлетенів,"2.К-сть виборців, внесених до списку","3.К-сть виборців, внесених до витягу для голос. за місцем перебув.",4.К-сть невикор. бюлетенів,"5.К-сть виборців, які отримали бюлетені у приміщ.","6.К-сть виборців, які отримали бюлетені за місцем перебув.",...,РиговановРуслан,СкоцикВіталій,СмешкоІгор,СоловйовОлександр,ТарутаСергій,ТимошенкоЮлія,ТимошенкоЮрій,ШевченкоІгор,ШевченкоОлександр,Дата та час протоколу
0,050130,11,"Вінницька область, Територіальний виборчий окр...","Школа-ліцей, спортивна зала",1704,1686,112,583,1031,90,...,1,1,120,0,0,169,11,1,3,01.04.2019 01:30
1,050131,11,"Вінницька область, Територіальний виборчий окр...","ДНЗ ""Казка"", спортивна зала",2148,2109,17,790,1348,10,...,0,2,147,0,2,213,19,0,9,01.04.2019 00:50
2,050132,11,"Вінницька область, Територіальний виборчий окр...","ПСП ""Агрофірма Батьківщина"", приіщення контори",1294,1282,4,453,837,4,...,0,0,95,2,0,104,9,1,1,02.04.2019 23:45
3,050133,11,"Вінницька область, Територіальний виборчий окр...","школа, спортзал",2439,2434,15,845,1582,12,...,0,3,186,0,0,231,10,1,12,01.04.2019 19:00
4,050134,11,"Вінницька область, Територіальний виборчий окр...","звіргосподарство, кабінет головного бухгалтера",222,225,0,82,140,0,...,0,0,14,0,0,26,0,1,0,01.04.2019 00:10


In [69]:
df_all.to_csv('elections_round1_v1.csv',index=False)